# Homework 6

In this homework you will be training and using a "char-RNN". This is the name given to a character-level recurrent neural network language model by [this famous blog post by Andrej Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Before you start on the rest of the homework, please give the blog post a read, it's quite good!

I don't expect you to implement the char-RNN from scratch. Andrej's original char-rnn is in Torch (the predecessor to PyTorch that is not commonly used anymore). Fortunately, there are many other implementations of this model available; for example, there is one (in both mxnet and pytorch) in chapters 8 and 9 of [the textbook](http://d2l.ai), and another pytorch one [here](https://github.com/spro/char-rnn.pytorch). **Please use one of these example implementations (or another one that you find) when completing this homework**.

For this homework, please complete the following steps:

1. Download and tokenize the [Shakespeare dataset](http://www.gutenberg.org/files/100/100-0.txt) at a character level. I recommend basing your solution on the following code:
```Python
# Remove non-alphabetical characters, lowercase, and replace whitespace with ' '
raw_dataset = ' '.join(re.sub('[^A-Za-z ]+', '', text).lower().split())
# Maps token index to character
idx_to_char = list(set(raw_dataset))
# Maps character to token index
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
# Tokenize the dataset
corpus_indices = [char_to_idx[char] for char in raw_dataset]
```
1. Train a "vanilla" RNN (as described in chapter 9 of [the textbook](http://d2l.ai)) on the Shakespeare dataset. Report the training loss and generate some samples from the model at the end of training.
1. Train a GRU RNN (as described in chapter 10 of [the textbook](http://d2l.ai)) on the Shakespeare datatset. Is the final training loss higher or lower than the vanilla RNN? Are the samples from the model more or less realistic?
1. Find a smaller, simpler dataset than the Shakespeare data (you can find some ideas in Andrej's blog post, but feel free to get creative!) and train either the vanilla or GRU RNN on it instead. Is the final training loss higher or lower than it was for the Shakespeare data?

1) data is tokenized on following cells steps (2-4)

In [5]:
!pip install Unidecode

imports

In [6]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import unidecode
import string
import random
import time
import math
import os
import argparse
import requests
from tqdm import tqdm

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


2) Vanilla RNN trained on Shakespeare ([source](https://github.com/vnikme/char-rnn.pytorch/tree/master))

In [13]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, model="gru", n_layers=1):
        super(CharRNN, self).__init__()
        self.model = model.lower()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(input_size, hidden_size)
        if self.model == "gru":
            self.rnn = nn.GRU(hidden_size, hidden_size, n_layers, batch_first=True)
        elif self.model == "lstm":
            self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers, batch_first=True)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        """
            input: shape=(batch_size, seq_size)
            output: shape=(batch_size, seq_size, output_size)
        """
        encoded = self.encoder(input)
        output, hidden = self.rnn(encoded, hidden)
        output = self.decoder(output)
        return output, hidden

    def init_hidden(self, batch_size, cuda):
        cuda_wrapper = lambda x: x.cuda() if cuda else x
        if self.model == "lstm":
            return (cuda_wrapper(Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))),
                    cuda_wrapper(Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))))
        return cuda_wrapper(Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)))

In [15]:
import string
import random
import time
import math

def read_file(filename):
    file = open(filename).read()
    all_characters = list(set(file))
    return file, len(file), all_characters, len(all_characters)

# Turning a string into a tensor

def char_tensor(string, all_characters):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = all_characters.index(string[c])
        except:
            continue
    return tensor

# Readable time elapsed

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [16]:

import os
import argparse

def generate(decoder, all_characters, prime_str='A', predict_len=100, temperature=0.8, cuda=False):
    hidden = decoder.init_hidden(1, cuda)
    prime_input = Variable(char_tensor(prime_str, all_characters).unsqueeze(0))

    if cuda:
        prime_input = prime_input.cuda()
    predicted = prime_str

    # Use priming string to "build up" hidden state
    _, hidden = decoder(prime_input, hidden)
    inp = prime_input[0,-1].view(1, -1)

    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)

        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]

        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = Variable(char_tensor(predicted_char, all_characters).unsqueeze(0))
        if cuda:
            inp = inp.cuda()

    return predicted

In [19]:
filename = '/content/drive/MyDrive/CS2516/Homework/shakespeare.txt'
model = 'lstm'
n_epochs = 2000
print_every = 100
hidden_size = 128
n_layers = 2
learning_rate = 0.01
chunk_len = 200
batch_size = 128
shuffle = True
cuda = True

if cuda:
    print("Using CUDA")

file, file_len, all_characters, n_characters = read_file(filename)

def random_training_set(chunk_len, batch_size):
    inp = torch.LongTensor(batch_size, chunk_len)
    target = torch.LongTensor(batch_size, chunk_len)
    for bi in range(batch_size):
        start_index = random.randint(0, file_len - chunk_len - 1)
        end_index = start_index + chunk_len + 1
        chunk = file[start_index:end_index]
        inp[bi] = char_tensor(chunk[:-1], all_characters)
        target[bi] = char_tensor(chunk[1:], all_characters)
    inp = Variable(inp)
    target = Variable(target)
    if cuda:
        inp = inp.cuda()
        target = target.cuda()
    return inp, target

def train(inp, target):
    """
        inp: (batch_size, seq_size)
        target: (batch_size, seq_size)
    """
    hidden = decoder.init_hidden(batch_size, cuda)
    decoder.zero_grad()

    output, hidden = decoder(inp, hidden)
    loss = criterion(output.view(-1, output.size(-1)), target.view(-1))

    loss.backward()
    decoder_optimizer.step()

    return loss.item()

def save():
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    torch.save((all_characters, decoder), save_filename)
    print('Saved as %s' % save_filename)

# Initialize models and start training

decoder = CharRNN(
    n_characters,
    hidden_size,
    n_characters,
    model=model,
    n_layers=n_layers,
)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

if cuda:
    decoder.cuda()

start = time.time()
all_losses = []
loss_avg = 0

try:
    print("Training for %d epochs..." % n_epochs)
    for epoch in tqdm(range(1, n_epochs + 1)):
        loss = train(*random_training_set(chunk_len, batch_size))
        loss_avg += loss

        if epoch % print_every == 0:
            print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
            print(generate(decoder, all_characters, 'Wh', 100, cuda=cuda), '\n')

    print("Saving...")
    save()

except KeyboardInterrupt:
    print("Saving before quit...")
    save()

Using CUDA
Training for 2000 epochs...


  5%|▌         | 100/2000 [00:25<08:24,  3.77it/s]

 25s (100 5%) 1.7850]
Wh semple the maid, in.

MENIUS:
And his king of 'tide to verdel's grumpet,
But prick you, to parged i 



 10%|█         | 200/2000 [00:50<08:16,  3.63it/s]

 50s (200 10%) 1.5559]
When, by that the male or meat:
The good:
let a kiships the ruins he made the might ture and earred bu 



...

 30s (1800 90%) 1.2686]
Where is a merciful up.

SICINIUS:
No, boy!

Lord:
Let me be the gates to the bear
To slaughter'd hope 



 95%|█████████▌| 1900/2000 [07:55<00:28,  3.55it/s]

 55s (1900 95%) 1.2576]
Where it be done: but that hitles may be executed,
And with him do will strive, become in it.

DUKE VI 



100%|██████████| 2000/2000 [08:20<00:00,  3.99it/s]

 20s (2000 100%) 1.2537]
Where have I have no more before I
Was neglicians, and to my love the flierful devil
To the Thrance an 

Saving...
Saved as shakespeare.pt


final loss:

`1.2537`

Final generated sequence:

`Where have I have no more before I
Was neglicians, and to my love the flierful devil
To the Thrance an`

3) GRU RNN trained on Shakespeare([source](https://github.com/vnikme/char-rnn.pytorch/tree/master))

In [17]:

filename = '/content/drive/MyDrive/CS2516/Homework/shakespeare.txt'
model = 'gru'
n_epochs = 2000
print_every = 100
hidden_size = 128
n_layers = 2
learning_rate = 0.01
chunk_len = 200
batch_size = 128
shuffle = True
cuda = True

if cuda:
    print("Using CUDA")

file, file_len, all_characters, n_characters = read_file(filename)

def random_training_set(chunk_len, batch_size):
    inp = torch.LongTensor(batch_size, chunk_len)
    target = torch.LongTensor(batch_size, chunk_len)
    for bi in range(batch_size):
        start_index = random.randint(0, file_len - chunk_len - 1)
        end_index = start_index + chunk_len + 1
        chunk = file[start_index:end_index]
        inp[bi] = char_tensor(chunk[:-1], all_characters)
        target[bi] = char_tensor(chunk[1:], all_characters)
    inp = Variable(inp)
    target = Variable(target)
    if cuda:
        inp = inp.cuda()
        target = target.cuda()
    return inp, target

def train(inp, target):
    """
        inp: (batch_size, seq_size)
        target: (batch_size, seq_size)
    """
    hidden = decoder.init_hidden(batch_size, cuda)
    decoder.zero_grad()

    output, hidden = decoder(inp, hidden)
    loss = criterion(output.view(-1, output.size(-1)), target.view(-1))

    loss.backward()
    decoder_optimizer.step()

    return loss.item()

def save():
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    torch.save((all_characters, decoder), save_filename)
    print('Saved as %s' % save_filename)

# Initialize models and start training

decoder = CharRNN(
    n_characters,
    hidden_size,
    n_characters,
    model=model,
    n_layers=n_layers,
)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

if cuda:
    decoder.cuda()

start = time.time()
all_losses = []
loss_avg = 0

try:
    print("Training for %d epochs..." % n_epochs)
    for epoch in tqdm(range(1, n_epochs + 1)):
        loss = train(*random_training_set(chunk_len, batch_size))
        loss_avg += loss

        if epoch % print_every == 0:
            print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
            print(generate(decoder, all_characters, 'Wh', 100, cuda=cuda), '\n')

    print("Saving...")
    save()

except KeyboardInterrupt:
    print("Saving before quit...")
    save()

Using CUDA
Training for 2000 epochs...


  5%|▌         | 100/2000 [00:23<08:31,  3.72it/s]

 23s (100 5%) 1.6169]
Whom Olar the blood
That your contleans of the at be the nerved let;
And sine, not son't the eath not  



 10%|█         | 200/2000 [00:48<07:57,  3.77it/s]

 48s (200 10%) 1.4490]
Whought,
Braken thee him actions, thou discore stames may from me,
Looks and for bope into enemy, and  



...

 17s (1800 90%) 1.2644]
Why, Gaunt is a woman's blind fear,
Which makes an oaths to protect a deputy,
In privateous comfort th 



 95%|█████████▌| 1900/2000 [07:42<00:25,  3.96it/s]

 42s (1900 95%) 1.3071]
Whose countrymen, and all, if the cords be gone.

KING LEWIS XI:
Take it those much you and for my mea 



100%|██████████| 2000/2000 [08:06<00:00,  4.11it/s]

 6s (2000 100%) 1.2759]
Why forward, not there.

HERMIONE:
He should rid him not stay my sight in the king;
You must come beho 

Saving...
Saved as shakespeare.pt


final loss:

`1.2759`

Final generated sequence:

`Why forward, not there.`

`HERMIONE:
He should rid him not stay my sight in the king;
You must come beho`

The final los is higher (but very close) when comapred to the previous model. Nevertheless, the generated sequences appear to have similar realism.

4) GRU RNN trained on dad jokes

In [18]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import argparse
import os

from tqdm import tqdm

filename = '/content/drive/MyDrive/CS2516/Homework/jokes.txt'
model = 'gru'
n_epochs = 2000
print_every = 100
hidden_size = 128
n_layers = 2
learning_rate = 0.01
chunk_len = 200
batch_size = 128
shuffle = True
cuda = True

if cuda:
    print("Using CUDA")

file, file_len, all_characters, n_characters = read_file(filename)

def random_training_set(chunk_len, batch_size):
    inp = torch.LongTensor(batch_size, chunk_len)
    target = torch.LongTensor(batch_size, chunk_len)
    for bi in range(batch_size):
        start_index = random.randint(0, file_len - chunk_len - 1)
        end_index = start_index + chunk_len + 1
        chunk = file[start_index:end_index]
        inp[bi] = char_tensor(chunk[:-1], all_characters)
        target[bi] = char_tensor(chunk[1:], all_characters)
    inp = Variable(inp)
    target = Variable(target)
    if cuda:
        inp = inp.cuda()
        target = target.cuda()
    return inp, target

def train(inp, target):
    """
        inp: (batch_size, seq_size)
        target: (batch_size, seq_size)
    """
    hidden = decoder.init_hidden(batch_size, cuda)
    decoder.zero_grad()

    output, hidden = decoder(inp, hidden)
    loss = criterion(output.view(-1, output.size(-1)), target.view(-1))

    loss.backward()
    decoder_optimizer.step()

    return loss.item()

def save():
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    torch.save((all_characters, decoder), save_filename)
    print('Saved as %s' % save_filename)

# Initialize models and start training

decoder = CharRNN(
    n_characters,
    hidden_size,
    n_characters,
    model=model,
    n_layers=n_layers,
)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

if cuda:
    decoder.cuda()

start = time.time()
all_losses = []
loss_avg = 0

try:
    print("Training for %d epochs..." % n_epochs)
    for epoch in tqdm(range(1, n_epochs + 1)):
        loss = train(*random_training_set(chunk_len, batch_size))
        loss_avg += loss

        if epoch % print_every == 0:
            print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
            print(generate(decoder, all_characters, 'Wh', 100, cuda=cuda), '\n')

    print("Saving...")
    save()

except KeyboardInterrupt:
    print("Saving before quit...")
    save()

Using CUDA
Training for 2000 epochs...


  5%|▌         | 100/2000 [00:25<08:06,  3.91it/s]

 24s (100 5%) 0.8921]
Whe was favorite no pression with a roman pizza tree?<> When he was the difference.
Have tay on the bo 



 10%|█         | 200/2000 [00:50<07:42,  3.89it/s]

 50s (200 10%) 0.1851]
Whatical fow do you go to read a batile?<>It was always so jaded.
Why did the coffee secroson?<> They' 



...

 23s (1800 90%) 0.0742]
When it gets bad, I take something for it.
I used to be addicted to soap...<> but I'm clean now.
When  



 95%|█████████▌| 1900/2000 [07:47<00:26,  3.80it/s]

 47s (1900 95%) 0.0727]
When it's ajar.
I made a belt out of watches once...<> It was a waist of time.
This furniture store ke 



100%|██████████| 2000/2000 [08:11<00:00,  4.07it/s]

 11s (2000 100%) 0.0767]
When it's ajar.
I made a belt out of watches once...<> It was a waist outates it saw the salad dressin 

Saving...
Saved as jokes.pt


The achieved loss is lower than the model trained on Shakespeare.

final loss:

`0.0767`

Final generated sequence:

`When it's ajar.
I made a belt out of watches once...<> It was a waist outates it saw the salad dressin `